## 最終課題


In [ ]:
import requests # HTTPリクエスト用ライブラリ
from bs4 import BeautifulSoup # HTML解析用ライブラリ
from urllib.parse import urljoin, urlparse
import time
import re

START_URL = "https://www.musashino-u.ac.jp/" #クロール開始URL

SLEEP_TIME = 0.5 #過負荷防止のための待機時間（秒）


sitemap_data = {} 
visited_urls = set()

def get_title(soup):
    """BeautifulSoupオブジェクトからページのタイトルを抽出する"""
    if soup.title and soup.title.string: #もし<title>タグが存在すればその内容を返す
        
        return soup.title.string.strip()
    return "TITLE NOT FOUND"

def is_valid_url(url, base_domain):
    """
    URLが同一ドメインに属し、クロール対象外のファイルではないかを確認する
    """
    try: # URLの解析
        parsed_url = urlparse(url)
    except ValueError:
        return False
    
    # スキームがhttpまたはhttpsであることを確認
    if parsed_url.scheme not in ['http', 'https']:
        return False
        
    # ドメインがベースドメインと一致することを確認
    if parsed_url.netloc != base_domain:
        return False
        
    # クロール対象外のファイル拡張子（画像、PDFなど）を除外
    excluded_extensions = (
        '.pdf', '.jpg', '.jpeg', '.png', '.gif', '.zip', '.mp4', '.css', '.js',
        '.ico', '.xml', '.rss', '.atom'
    )
    if any(url.lower().endswith(ext) for ext in excluded_extensions):
        return False
        
    return True

def crawl_site(url, base_domain):
    """
    指定されたURLをクロールし、同一ドメイン内の未訪問のリンクを再帰的に辿る
    """
    if url in visited_urls:
        return

    print(f"--- Visiting: {url} ---")
    visited_urls.add(url)
    
    try:
        # Webサイトにアクセス
        response = requests.get(url, timeout=10)
        # 正常なステータスコードを確認
        response.raise_for_status() 
        # エンコーディングの指定
        response.encoding = response.apparent_encoding

        # BeautifulSoupでHTMLを解析
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # タイトルを抽出して辞書に格納
        title = get_title(soup)
        sitemap_data[url] = title
        print(f"    Title: {title}")

        # ページ内の全てのリンクを検索
        for link in soup.find_all('a', href=True):
            href = link.get('href')
            # 絶対URLに変換
            absolute_url = urljoin(url, href)
            # URLからフラグメント（#...）を除去
            absolute_url_no_fragment = absolute_url.split('#')[0]

            # 同一ドメイン内で有効なURLかチェック
            if is_valid_url(absolute_url_no_fragment, base_domain): 
                # 未訪問のURLであれば再帰的にクロール
                if absolute_url_no_fragment not in visited_urls: 
                    print(f"    Found New Link: {absolute_url_no_fragment}")
                    
                    # 負荷軽減のための待機
                    time.sleep(SLEEP_TIME) 
                    
                    # 再帰呼び出し
                    crawl_site(absolute_url_no_fragment, base_domain)
                    
    except requests.exceptions.RequestException as e:
        # アクセスエラー（404, 500, タイムアウトなど）を捕捉
        print(f" Error accessing {url}: {e}")
    except Exception as e:
        # その他予期せぬエラー
        print(f" An unexpected error occurred at {url}: {e}")


if __name__ == "__main__":
    
    base_domain = urlparse(START_URL).netloc
    
    print(f"--- クロール開始 ---")
    print(f"ターゲットドメイン: **{base_domain}**")
    print(f"待機時間: {SLEEP_TIME}秒/ページ\n")
    
    # クロール処理の開始
    crawl_site(START_URL, base_domain)
    
    print("\n--- クロール完了 ---")
    
    # 結果の表示
    print(f"\n抽出されたサイトマップデータ ({len(sitemap_data)} ページ):\n")
    print("{\n" + ",\n".join(f'    "{url}": "{title}"' for url, title in sitemap_data.items()) + "\n}")

--- クロール開始 ---
ターゲットドメイン: **www.musashino-u.ac.jp**
待機時間: 0.5秒/ページ

--- Visiting: https://www.musashino-u.ac.jp/ ---
    Title: 武蔵野大学
    Found New Link: https://www.musashino-u.ac.jp/access.html
--- Visiting: https://www.musashino-u.ac.jp/access.html ---
    Title: 交通アクセス | 武蔵野大学
    Found New Link: https://www.musashino-u.ac.jp/admission/request.html
--- Visiting: https://www.musashino-u.ac.jp/admission/request.html ---
    Title: 資料請求 | 入試情報 | 武蔵野大学
    Found New Link: https://www.musashino-u.ac.jp/contact.html
--- Visiting: https://www.musashino-u.ac.jp/contact.html ---
    Title: お問い合わせ | 武蔵野大学
    Found New Link: https://www.musashino-u.ac.jp/prospective-students.html
--- Visiting: https://www.musashino-u.ac.jp/prospective-students.html ---
    Title: 武蔵野大学で学びたい方 | 武蔵野大学
    Found New Link: https://www.musashino-u.ac.jp/students.html
--- Visiting: https://www.musashino-u.ac.jp/students.html ---
    Title: 在学生の方 | 武蔵野大学
    Found New Link: https://www.musashino-u.ac.jp/alumni.html

KeyboardInterrupt: 